In [1]:
!pip install datasets

In [2]:
import wandb
wandb.login(key="2bcea92e3d3f472ffce763d1296a2ff409c4662a")
from tqdm import tqdm


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: m-tannistha. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import argparse
import random
import numpy as np
from functools import partial
from torch.cuda.amp import autocast, GradScaler


import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW

from torch.utils.data import DataLoader
from datasets import load_dataset
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = False

In [4]:
def seed_everything(seed=2003):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def calculate_DPO_loss(model_prefered_logprob, model_disprefered_logprob,
                       ref_prefered_logprob, ref_disprefered_logprob,
                       beta=0.5):

    prefered_relative_logprob = model_prefered_logprob - ref_prefered_logprob
    disprefered_relative_logprob = model_disprefered_logprob - ref_disprefered_logprob

    reward_accuracies = (prefered_relative_logprob > disprefered_relative_logprob).float().mean(dim=-1)
    reward_margins = (prefered_relative_logprob - disprefered_relative_logprob).mean(dim=-1)

    loss = -F.logsigmoid(beta * (prefered_relative_logprob - disprefered_relative_logprob)).mean(dim=-1)

    return loss, prefered_relative_logprob.mean(dim=-1), disprefered_relative_logprob.mean(dim=-1), reward_accuracies, reward_margins

def get_log_prob(logits, labels):
    log_probs = F.log_softmax(logits, dim=-1)
    return torch.gather(log_probs, -1, labels.unsqueeze(-1)).squeeze(-1).mean(-1)

def collate_fn(batch, tokenizer, max_length, device):
    prompts = ['Instruct: ' + item['prompt'] + '\n' for item in batch]
    chosen_responses = ['Output: ' + item['chosen'] for item in batch]
    rejected_responses = ['Output: ' + item['rejected'] for item in batch]

    prompt_ids = tokenizer.batch_encode_plus(prompts, padding=True, return_tensors="pt", max_length=max_length, truncation=True)['input_ids'].to(device)
    prefered_ids = tokenizer.batch_encode_plus(chosen_responses, padding=True, return_tensors="pt", max_length=max_length, truncation=True)['input_ids'].to(device)
    disprefered_ids = tokenizer.batch_encode_plus(rejected_responses, padding=True, return_tensors="pt", max_length=max_length, truncation=True)['input_ids'].to(device)

    prompt_prefered_ids = torch.cat([prompt_ids, prefered_ids], dim=-1)
    prompt_disprefered_ids = torch.cat([prompt_ids, disprefered_ids], dim=-1)

    prompt_prefered_mask = torch.cat([torch.ones_like(prompt_ids), torch.zeros_like(prefered_ids)], dim=-1)
    prompt_disprefered_mask = torch.cat([torch.ones_like(prompt_ids), torch.zeros_like(disprefered_ids)], dim=-1)

    return {'prompt_prefered_ids': prompt_prefered_ids,
            'prompt_disprefered_ids': prompt_disprefered_ids,
            'prompt_prefered_mask': prompt_prefered_mask,
            'prompt_disprefered_mask': prompt_disprefered_mask}

def train(model, ref_model, tokenizer, optimizer, train_dataloader, epochs=1, beta=0.1):
    model.train()
    ref_model.eval()
    model, train_dataloader = accelerator.prepare(model, train_dataloader)

    for batch in train_dataloader:
       # Check and print tensor devices for batch tensors
      print("Batch devices:")
      print("prompt_prefered_ids device:", batch['prompt_prefered_ids'].device)
      print("prompt_disprefered_ids device:", batch['prompt_disprefered_ids'].device)

      # Move model tensors to the GPU (handled by Accelerator)
      prompt_prefered_ids = batch['prompt_prefered_ids']
      prompt_disprefered_ids = batch['prompt_disprefered_ids']
      prompt_prefered_mask = batch['prompt_prefered_mask']
      prompt_disprefered_mask = batch['prompt_disprefered_mask']

      # Check devices after moving to device (already handled by Accelerator)
      print("After moving to device:")
      print("prompt_prefered_ids device:", prompt_prefered_ids.device)
      print("prompt_disprefered_ids device:", prompt_disprefered_ids.device)

      # Forward pass with main model (on GPU)
      model_prefered_log_prob = get_log_prob(model(prompt_prefered_ids, attention_mask=prompt_prefered_mask).logits.to(device),prompt_prefered_ids)
      model_disprefered_log_prob = get_log_prob(model(prompt_disprefered_ids, attention_mask=prompt_disprefered_mask).logits.to(device),prompt_disprefered_ids)
      print("Before loss calculation:")

      # Move tensors to CPU for ref_model forward pass if required
      prompt_prefered_ids_cpu = prompt_prefered_ids.to("cpu")
      prompt_disprefered_ids_cpu = prompt_disprefered_ids.to("cpu")
      prompt_prefered_mask_cpu = prompt_prefered_mask.to("cpu")
      prompt_disprefered_mask_cpu = prompt_disprefered_mask.to("cpu")

      # Forward pass with ref_model (on CPU)
      with torch.no_grad():
          ref_prefered_log_prob = get_log_prob(
              ref_model(prompt_prefered_ids_cpu, attention_mask=prompt_prefered_mask_cpu).logits,
              prompt_prefered_ids_cpu
          )
          ref_disprefered_log_prob = get_log_prob(
              ref_model(prompt_disprefered_ids_cpu, attention_mask=prompt_disprefered_mask_cpu).logits,
              prompt_disprefered_ids_cpu
          )

      # Move ref_model outputs to GPU for loss calculation
      ref_prefered_log_prob = ref_prefered_log_prob.to(device)
      ref_disprefered_log_prob = ref_disprefered_log_prob.to(device)

      # Check that all tensors are now on the same device before loss calculation
      print("Before loss calculation:")
      print("model_prefered_log_prob device:", model_prefered_log_prob.device)
      print("model_disprefered_log_prob device:", model_disprefered_log_prob.device)
      print("ref_prefered_log_prob device:", ref_prefered_log_prob.device)
      print("ref_disprefered_log_prob device:", ref_disprefered_log_prob.device)

      # Calculate DPO loss on GPU
      loss, prefered_relative_logprob, disprefered_relative_logprob, reward_accuracies, reward_margins = calculate_DPO_loss(
          model_prefered_log_prob, model_disprefered_log_prob,
          ref_prefered_log_prob, ref_disprefered_log_prob,
          beta=beta
      )

    # Backpropagation and optimization
    loss.backward()
    optimizer.step()

In [5]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install --upgrade transformers accelerate bitsandbytes

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer.pad_token = tokenizer.eos_token
model = model.to(device)
ref_model = model.to("cpu")
optimizer = AdamW(model.parameters(), lr=1e-6)

dataset = load_dataset("jondurbin/truthy-dpo-v0.1", split="train[:100]")
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=partial(collate_fn, tokenizer=tokenizer, max_length=512, device=device))

train(model, ref_model, tokenizer, optimizer, train_dataloader, epochs=10, beta=0.1)

Batch devices:
prompt_prefered_ids device: cuda:0
prompt_disprefered_ids device: cuda:0
After moving to device:
prompt_prefered_ids device: cuda:0
prompt_disprefered_ids device: cuda:0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)